# 1. Import packages and setup environment (CPU/GPU/TPU)

In [ ]:
## For TPU environment (install missing packages / reinstall tensorflow to solve NaN topic during training / restart kernel)

import IPython
import tensorflow as tf
IPython.display.clear_output() # Workaround for error messages leading to Failed notebook

if len(tf.config.experimental.list_logical_devices('TPU')) > 0:
    !pip install -q tensorflow-tpu -f https://storage.googleapis.com/libtpu-tf-releases/index.html --force-reinstall
    !pip install -q pydot
    !pip install -q -U keras-tuner
    !pip install -q polars
    !pip install -q pydicom
    !pip install -q protobuf==5.29.5 # to solve tuner compatibility issue
    IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
## Import packages

# General purpose modules
import os
import math
from tqdm import tqdm
import time
from pathlib import Path
from natsort import natsorted
import cv2

# Data handling and visualization modules
import json
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

# Skikit-learn preprocessing modules
from sklearn.model_selection import StratifiedKFold

# Tensorflow modules
import tensorflow as tf
from tensorflow.keras import backend as K
import keras_tuner as kt

In [ ]:
## Detect hardware (CPU/GPU/TPU), setup environment and return appropriate distribution strategy

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect(tpu='local') # set tpu is local as it should be available in the VM
    print('✅ Running on TPU ', tpu.master())
except:
    print('❌ Using CPU/GPU')
    tpu = None

if tpu:
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

# 2. Load and explore data

In [ ]:
## Preprocessing functions

image_size = 256 # input image size fo neural network model
CLASSES = {0 : 'autentic', 1: 'forged'}

# Pad and resize images (while retaining aspect ratio) and adjust coordinates accordingly
def pad_and_resize(image):
    image_size_rows, image_size_cols, _ = image.shape
    pad_size = max(image_size_rows, image_size_cols)
    image_padded = tf.image.resize_with_crop_or_pad(image, pad_size, pad_size)
    image_resized = tf.image.resize(image_padded, [image_size, image_size])
    return image_resized

# Zoom/rotate/translate images and adjust coordinates accordingly
def image_augmentation(image, augmentation=True):
    if augmentation:
        zoom_fac = np.random.uniform(0.0, 0.0)
        rot_fac = np.random.uniform(-0.1, 0.1)
        trans_fac = np.random.uniform(-0.05, 0.05)
        z = tf.keras.layers.RandomZoom(height_factor=(zoom_fac, zoom_fac), fill_mode='constant', name='auglay1')(image)
        z = tf.keras.layers.RandomRotation(factor=(rot_fac, rot_fac), fill_mode='constant', name='auglay2')(z)
        image = tf.keras.layers.RandomTranslation(height_factor=(trans_fac, trans_fac), width_factor=(trans_fac, trans_fac),
                                                  interpolation='nearest', fill_mode='constant', name='auglay3')(z)
    return image

# Preprocess image or masking (Padding and resizing to image_size x image_size x channel)
def preprocess_images(image, augmentation=False):
    image_scaled = image.astype(dtype=np.float32)/255
    image_aug = image_augmentation(image_scaled, augmentation=augmentation)
    image_resized = pad_and_resize(image_aug)
    image_resized = tf.cast(image_resized*255, dtype=tf.uint8)
    return image_resized

# Postprocess image or masking (Resizing and croping to original image/mask size)
def postprocess_images(image, orig_image):
    image_scaled = tf.cast(image, dtype=np.float32)/255
    image_size_rows, image_size_cols, _ = orig_image.shape
    max_orig_size = max(image_size_rows, image_size_cols)
    image_resized = tf.image.resize(image_scaled, [max_orig_size, max_orig_size])
    image_padded = tf.image.resize_with_crop_or_pad(image_resized, image_size_rows, image_size_cols)
    image_rescaled = tf.cast(image_padded*255, dtype=tf.uint8)
    return image_rescaled

In [ ]:
## Load and preprocess images

SUBMISSIONING = False
folder_path_au = Path("/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images/authentic")
folder_path_fo = Path("/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images/forged")

def load_images(folder_path):
    images = []
    labels = []
    for file_path in tqdm(folder_path.glob("*.png")):
        image = cv2.imread(str(file_path))
        if 'authentic' in str(folder_path):
            label = np.zeros((image.shape[0], image.shape[1], 1))
        else:
            mask_filename = str(file_path).split('/')[-1].replace('.png', '.npy')
            mask_raw = np.load('/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_masks/' + mask_filename)
            label = np.transpose(mask_raw, axes=[1,2,0])
            label = np.sum(label, axis=2, keepdims=True)
        if image is not None:
            image = preprocess_images(image, False)
            label = preprocess_images(label, False)
            images.append(image)
            labels.append(label)
    images = np.stack(images, axis=0)
    labels = np.stack(labels, axis=0)
    return images, labels

def load_dataset():
    images_au, labels_au = load_images(folder_path_au)
    images_fo, labels_fo = load_images(folder_path_fo)
    images = np.concatenate((images_au, images_fo), axis=0)
    labels = np.concatenate((labels_au, labels_fo), axis=0)
    return images, labels

if not SUBMISSIONING:
    trainval_images, trainval_labels = load_dataset()
else: # Dummy data for speeding up submission
    trainval_images = np.ones((640,256,256,3))
    trainval_labels = np.zeros((640,256,256,1))

In [ ]:
## Spliting trainval data into train and validation data with StratifiedKFold

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42) # Baseline 42
bool_labels = (trainval_labels.sum(axis=(1,2,3))>0).astype(dtype=np.float32)
for fold, (train_idx, val_idx) in enumerate(skf.split(trainval_images, y=bool_labels)):
    train_images, val_images = trainval_images[train_idx], trainval_images[val_idx]
    train_labels, val_labels = trainval_labels[train_idx], trainval_labels[val_idx]
    print(f"✅ Fold {fold}: Train size = {len(train_idx)}, Val size = {len(val_idx)}")
    break  # Use only the first fold for now

In [ ]:
## Zoom/rotate/translate images and adjust labels accordingly (not yet integrated)

def image_augmentation_ds(image, label, augmentation=True):
    if augmentation:
        zoom_fac = np.random.uniform(-0.1, 0.1)
        rot_fac = np.random.uniform(-0.1, 0.1)
        trans_fac = np.random.uniform(-0.05, 0.05)
        
        x = tf.keras.layers.RandomZoom(height_factor=(zoom_fac, zoom_fac), fill_mode='constant', name='auglay1')(image)
        x = tf.keras.layers.RandomRotation(factor=(rot_fac, rot_fac), fill_mode='constant', name='auglay2')(x)
        image = tf.keras.layers.RandomTranslation(height_factor=(trans_fac, trans_fac), width_factor=(trans_fac, trans_fac),
                                                  interpolation='nearest', fill_mode='constant', name='auglay3')(x)
        
        y = tf.keras.layers.RandomZoom(height_factor=(zoom_fac, zoom_fac), fill_mode='constant', name='auglay1')(label)
        y = tf.keras.layers.RandomRotation(factor=(rot_fac, rot_fac), fill_mode='constant', name='auglay2')(y)
        label = tf.keras.layers.RandomTranslation(height_factor=(trans_fac, trans_fac), width_factor=(trans_fac, trans_fac),
                                              interpolation='nearest', fill_mode='constant', name='auglay3')(y)
    return image, label

In [ ]:
## Create train and validation datasets

SEED=42
batch_size=32
batch_size_val=32

train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_ds = train_ds.shuffle(len(train_labels), seed=SEED).repeat().batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE) #map(lambda image, label: image_augmentation_ds(image, label)).
val_ds = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_ds = val_ds.shuffle(len(val_labels)).batch(batch_size_val, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

print('Size of train dataset: '+ str(len(train_labels)))
print('Number of batches in train dataset: '+ f'{len(train_labels)//batch_size}')
print('Size of validation dataset: '+ str(len(val_labels)))
print('Number of batches in val dataset: '+ f'{len(val_labels)//batch_size_val}')

In [ ]:
## Check validation dataset batch dimensions

for X, y in val_ds.take(1):
    print(X.shape)
    print(y.shape)

# 3. Explore Data

In [ ]:
## Visualize data

train_ds_vis = train_ds.unbatch() #.shuffle(2048,seed=43)
num_examples = 36
num_columns = 6
num_rows = math.ceil(num_examples/num_columns)
plt.figure(figsize=(16, 16))
for i, (image, label) in enumerate(train_ds_vis.take(num_examples)):
    if i == -1: # Set to 0 in case of interest
        print(image.shape)
        print('class id: '+str(label.numpy()))
        print('class name: '+str(CLASSES[label.numpy()]))
    bool_label = (label.numpy().sum()>0).astype(dtype=np.float32)
    class_id = str(bool_label)
    class_name = str(CLASSES[bool_label])
    plt.subplot(num_rows, num_columns, i + 1)
    plt.imshow(image)
    if bool_label:
        mask = np.ma.masked_where(label == 0, label*255)
        plt.imshow(mask, cmap='Set1', alpha=0.5)
    plt.title(f"{class_name}({class_id})", fontsize=10)
    plt.suptitle("Examples from train dataset")
    plt.xticks([])
    plt.yticks([])

# 4. Build and explore neural network

In [ ]:
## Custom F1 function to handle imbalanced label classes

def f1_score(y_true, y_pred):
    y_pred = tf.round(y_pred)  # Round predictions to 0 or 1
    tp = K.sum(K.cast(y_true * y_pred, 'float'), axis=0)  # True positives
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float'), axis=0)  # False positives
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float'), axis=0)  # False negatives

    precision = tp / (tp + fp + K.epsilon())  # Precision calculation
    recall = tp / (tp + fn + K.epsilon())  # Recall calculation

    f1 = 2 * precision * recall / (precision + recall + K.epsilon())  # F1 score
    return K.mean(f1)

In [ ]:
## Custom F1 class to handle imbalanced label classes

@tf.keras.utils.register_keras_serializable()
class CustomF1(tf.keras.metrics.Metric):
    def __init__(self, name='cf1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.f1_score_fn = f1_score
        self.total = self.add_weight(shape=(), name="total", initializer="zeros")
        self.count = self.add_weight(shape=(), name="count", initializer="zeros")
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(tf.reshape(y_true, shape=(y_true.shape[0], -1)), dtype=tf.float32)
        y_pred = tf.reshape(y_pred, shape=(y_pred.shape[0], -1))
        metric = self.f1_score_fn(y_true, y_pred)
        self.total.assign_add(metric)
        self.count.assign_add(tf.cast(1, tf.float32))
    def result(self):
        return self.total / self.count

In [ ]:
## Weighted BCE function to handle imbalanced label classes

true_freq = train_labels.sum()/np.size(train_labels)

@tf.keras.utils.register_keras_serializable()
def weighted_binary_crossentropy(y_true, y_pred, zero_weight=true_freq, one_weight=1-true_freq):
    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = tf.cast(y_pred, dtype=tf.float32)
    # Clip predictions to avoid log(0)
    epsilon = K.epsilon()
    y_pred = K.clip(y_pred, epsilon, 1 - epsilon)
    
    # Compute binary cross-entropy
    bce = -(y_true * K.log(y_pred) + (1 - y_true) * K.log(1 - y_pred))
    
    # Apply weights
    weights = y_true * one_weight + (1 - y_true) * zero_weight
    weighted_bce = weights * bce
    return K.mean(weighted_bce)

The base network is a version of EfficientNet architecture (https://arxiv.org/pdf/1905.11946). The version number and the number of layers to retune are hyperparameters and can be tuned accordingly.

![](https://1.bp.blogspot.com/-Cdtb97FtgdA/XO3BHsB7oEI/AAAAAAAAEKE/bmtkonwgs8cmWyI5esVo8wJPnhPLQ5bGQCLcBGAs/s1600/image4.png)

In [ ]:
## Network configuration and preprocessing layer

# Network configurations
base_network = {4: "enb0",     # EfficientNetB0
                5: "enb1",     # EfficientNetB1
                6: "enb2",     # EfficientNetB2
                7: "enb3",     # EfficientNetB3
                8: "enb4",     # EfficientNetB4
                9: "env2b0",   # EfficientNetV2B0
                10: "env2b1",  # EfficientNetV2B1
                11: "env2b2",  # EfficientNetV2B2
                12: "env2b3",  # EfficientNetV2B3
                13: "env2s"}   # EfficientNetV2S

# Custom layer for preprocessing
@tf.keras.utils.register_keras_serializable()
class Rescale(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(Rescale, self).__init__(**kwargs)
    def call(self, inputs):
        x = tf.cast(inputs, tf.float32)/255
        return x

@tf.keras.utils.register_keras_serializable()
class PreProcess(tf.keras.layers.Layer):
    def __init__(self, base_network_type, **kwargs):
        super(PreProcess, self).__init__(**kwargs)
        if base_network_type < 9: self.preprocess_input = tf.keras.applications.efficientnet.preprocess_input
        elif base_network_type >= 9: self.preprocess_input = tf.keras.applications.efficientnet_v2.preprocess_input
        else: print('Wrong base network number have been choosen!!!')
    def call(self, inputs):
        return self.preprocess_input(inputs*255.0)

In [ ]:
def upsample(filters, size, norm_type='batchnorm', apply_dropout=False, name=None):
  """ Upsamples an input: Conv2DTranspose => Batchnorm => Dropout => Relu
      Args:
        filters: number of filters
        size: filter size
        norm_type: Normalization type; either 'batchnorm' or 'instancenorm'.
        apply_dropout: If True, adds the dropout layer
    
      Returns:
        Upsample Sequential Model"""

  initializer = tf.random_normal_initializer(0., 0.02)
  result = tf.keras.Sequential(name=name)
  result.add(
      tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                      padding='same',
                                      kernel_initializer=initializer,
                                      use_bias=False))

  if norm_type.lower() == 'batchnorm':
    result.add(tf.keras.layers.BatchNormalization())
  elif norm_type.lower() == 'instancenorm':
    result.add(InstanceNormalization())

  if apply_dropout:
    result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU())
  return result

In [ ]:
    layer_names = ['block1b_add',          # 128x128x24
                   'block2d_add',          # 64x64x32
                   'block3d_add',          # 32x32x56
                   #'block4f_add',          # 16x16x112
                   'block5f_add',          # 16x16x160
                   'block6h_add',          # 8x8x272
                   'block7b_add',          # 8x8x448
                   'top_activation',       # 8x8x1792
                   ]
    base_model_outputs = [base_model.get_layer(name).output for name in layer_names]

In [ ]:
base_model_outputs

In [ ]:
## Build UNet Architecture with pre-trained EfficientNet encoder and pix2pix decoder

def build_network(hp):
    # Select base model and corresponding preprocessing
    base_network_type = 8 #hp.Int(name='base_network_type', min_value=4, max_value=13, step=1, default=7) # Choose Pretrained Network
    if base_network_type == 4: base_model = tf.keras.applications.EfficientNetB0(
        include_top=False, input_shape=[image_size, image_size, 3], weights='imagenet')
    elif base_network_type == 5: base_model = tf.keras.applications.EfficientNetB1(
        include_top=False, input_shape=[image_size, image_size, 3], weights='imagenet')
    elif base_network_type == 6: base_model = tf.keras.applications.EfficientNetB2(
        include_top=False, input_shape=[image_size, image_size, 3], weights='imagenet')
    elif base_network_type == 7: base_model = tf.keras.applications.EfficientNetB3(
        include_top=False, input_shape=[image_size, image_size, 3], weights='imagenet')
    elif base_network_type == 8: base_model = tf.keras.applications.EfficientNetB4(
        include_top=False, input_shape=[image_size, image_size, 3], weights='imagenet')
    elif base_network_type == 9: base_model = tf.keras.applications.EfficientNetV2B0(
        include_top=False, input_shape=[image_size, image_size, 3], weights='imagenet')
    elif base_network_type == 10: base_model = tf.keras.applications.EfficientNetV2B1(
        include_top=False, input_shape=[image_size, image_size, 3], weights='imagenet')
    elif base_network_type == 11: base_model = tf.keras.applications.EfficientNetV2B2(
        include_top=False, input_shape=[image_size, image_size, 3], weights='imagenet')
    elif base_network_type == 12: base_model = tf.keras.applications.EfficientNetV2B3(
        include_top=False, input_shape=[image_size, image_size, 3], weights='imagenet')
    elif base_network_type == 13: base_model = tf.keras.applications.EfficientNetV2S(
        include_top=False, input_shape=[image_size, image_size, 3], weights='imagenet')
    else: print('Wrong base network number have been choosen!!!')
    prepocessing = PreProcess(base_network_type=base_network_type, name='preprocessing')

    # Choose base model layers to be trained
    base_model.trainable = False # freeze base model layers
    max_layer_nr = len(base_model.layers)
    # (B0): all:238 / 2ab+:220 / 3ab+:191 / 4abc+:162 / 5abc+:118 / 6abcd+:75 / 7a+:16
    # (B3): all:385 / 2ab+:355 / 3ab+:311 / 4abc+:267 / 5abc+:193 / 6abcd+:120 / 7a+:31
    layer_id = 16 #hp.Choice(name='layer_id', values=[355, max_layer_nr, 311, 267, 193, 120, 31]) # layer number from the network shall be trained
    print('Unfreeze base model layers from layer ' + str(base_model.layers[-layer_id]))
    for layer in base_model.layers[-layer_id:]: # unfreeze choosen layers
        layer.trainable = True

    # Create the feature extraction model
    layer_names = ['block1b_add',          # 128x128x24
                   'block2d_add',          # 64x64x32
                   'block3d_add',          # 32x32x56
                   #'block4f_add',          # 16x16x112
                   'block5f_add',          # 16x16x160
                   #'block6h_add',          # 8x8x272
                   #'block7b_add',          # 8x8x448
                   'top_activation',       # 8x8x1792
                   ]
    base_model_outputs = [base_model.get_layer(name).output for name in layer_names]
    down_stack = tf.keras.Model(inputs=base_model.input, outputs=base_model_outputs, name='downsampling')

    # Create the upsampling model
    up_stack = [upsample(160, 3, name='upsampling_block1'),  # 8x8 -> 16x16
                upsample(56, 3, name='upsampling_block2'),  # 16x16 -> 32x32
                upsample(32, 3, name='upsampling_block3'),  # 32x32 -> 64x64
                upsample(24, 3, name='upsampling_block4'),   # 64x64 -> 128x128
               ]

    # define the sets of inputs
    input_img = tf.keras.Input(shape=(image_size, image_size, 3), name='input_img')

    # Cast, rescale and preprocess image tensors
    x = Rescale(name='rescaling')(input_img)
    x = prepocessing(x)
    
    # Downsampling through the model
    skips = down_stack(x)
    x = skips[-1]
    skips = reversed(skips[:-1])

    # Upsampling and establishing the skip connections
    for i, (up, skip) in enumerate(zip(up_stack, skips)):
        x = up(x)
        x = tf.keras.layers.Concatenate(name=f'upsampling_concat{i+1}')([x, skip]) #Concatenate
        
    # This is the last layer of the model
    out = tf.keras.layers.Conv2DTranspose(filters=1, activation='sigmoid', kernel_size=3, strides=2, padding='same', name='conv2dtrans')(x) # 128x128 -> 256x256

    # define model
    model = tf.keras.Model(inputs=input_img, outputs=out, name='LUC_UNET')

    # define optimizer/loss and compile model
    lr_tune = 5e-4 #hp.Float(name='learning_rate', min_value=1e-4, max_value=1e-2, sampling='log', default=1e-3)
    optimizer = tf.keras.optimizers.Nadam(lr_tune)
    loss = weighted_binary_crossentropy #WeightedBinaryCrossentropy()
    metrics = CustomF1()
    model.compile(optimizer=optimizer, loss=loss, metrics=[metrics], run_eagerly=False)
    return model

if not SUBMISSIONING:
    with strategy.scope():
        model = build_network(kt.HyperParameters())
else:
    # Load pre-trained model for submission
    model = tf.keras.models.load_model('/kaggle/input/luc-1xx/luc_1_0_0.h5')
    print('Model weights have been loaded!')

In [ ]:
## Explore model architecture

model.summary(line_length=110)
# tf.keras.utils.plot_model(model, to_file='model_architecture.png', show_shapes=True, show_dtype=False,
#                           show_layer_names=True, show_layer_activations=True, show_trainable=False)

# 5. Training

In [ ]:
## Training parameters

epochs = 100
steps_per_epoch = len(train_labels)//batch_size
TUNING = False and not SUBMISSIONING
TRAINING = True and not SUBMISSIONING
FINETUNING = False and not SUBMISSIONING

In [ ]:
## Tuner configurations

if TUNING:
    i_TunerTyp = 1 # Choose desired tuner type: {1: 'grid', 2: 'random', 3: 'hyper'}
    TunerStr = {1: 'grid', 2: 'random', 3: 'hyper'}
    
    tuner_grid = kt.GridSearch(hypermodel=build_network, objective=kt.Objective("val_cf1_score", direction="max"),
                               max_trials=15, max_consecutive_failed_trials=1,
                               overwrite=True, directory="tuner", project_name="LUC", distribution_strategy = strategy)
    
    tuner_random = kt.RandomSearch(hypermodel=build_network, objective=kt.Objective("val_cf1_score", direction="max"),
                                   max_trials=10, executions_per_trial=1,
                                   overwrite=True, directory="tuner", project_name="LUC", distribution_strategy = strategy)
    
    tuner_hyper = kt.Hyperband(hypermodel=build_network, objective=kt.Objective("val_cf1_score", direction="max"),
                               max_epochs=60, factor=4, hyperband_iterations=1,
                               overwrite=True, directory="tuner", project_name="LUC", distribution_strategy = strategy)
    
    tuner = globals()[f'tuner_{TunerStr[i_TunerTyp]}']
    tuner.search_space_summary()

In [ ]:
## Train or tune model

# Callback functions
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=5, verbose=1, monitor='val_cf1_score', mode='max')
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, verbose=1, monitor='val_cf1_score', mode='max', restore_best_weights=True)
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-5 * 10**(epoch / 10)) # Find starting learning

# Training
if TRAINING or FINETUNING:
    history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, steps_per_epoch=steps_per_epoch,
                        callbacks=[lr_scheduler, early_stopping_cb])

# Tuning
if TUNING:
    tuner.search(train_ds, validation_data=val_ds, epochs=epochs, steps_per_epoch=steps_per_epoch,
                 callbacks=[lr_scheduler, early_stopping_cb])
    best_models = tuner.get_best_models(num_models=2)
    model = best_models[0]
    model.summary()
    tuner.results_summary()

In [ ]:
## Save weights of model after training/tuning/finetuning

if TRAINING or TUNING or FINETUNING:
    model.save('luc_1_1_0.h5', include_optimizer=False)
    print('Model weights have been saved!')

# 6. Evaluation

In [ ]:
## Plot learning curves

if TRAINING or FINETUNING:
    history_fil = {key: history.history[key] for key in ['cf1_score', 'val_cf1_score']}
    history_fil2 = {key: history.history[key] for key in ['loss', 'val_loss']}
    history_fil3 = {key: history.history[key] for key in ['learning_rate']}
    
    pd.DataFrame(history_fil).plot()
    plt.ylabel("Accuracy")
    plt.xlabel("epochs")
    pd.DataFrame(history_fil2).plot()
    plt.ylabel("Loss")
    plt.xlabel("epochs")
    #plt.axis([10, len(history_fil2['val_loss']), 0, history_fil2['val_loss'][10]+0.1*history_fil2['val_loss'][10]])
    pd.DataFrame(history_fil3).plot()
    plt.ylabel("Learning rate")

In [ ]:
## Compare predicted with ground true masks

def display(display_list):
    plt.figure(figsize=(15, 15))
    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.utils.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()

def create_mask(pred_mask):
    pred_mask = tf.cast(tf.math.greater(pred_mask, 0.5), dtype=tf.int8)
    return pred_mask[0]

def show_predictions(dataset=None, num=1):
    if dataset:
        for image, mask in dataset.take(num):
          pred_mask = model.predict(image, verbose=0)
          display([image[0], mask[0], create_mask(pred_mask)])
    else:
        display([sample_image, sample_mask,
                 create_mask(model.predict(sample_image[tf.newaxis, ...]))])

show_predictions(train_ds, 3)

# 7. Submission

In [ ]:
## Utility: RLE encode 

def _rle_one(arr):
    """Encode a single 2D binary mask into RLE list of pairs."""
    dots = np.where(arr.T.flatten() == 1)[0]
    if len(dots) == 0:
        return []
    run_lengths = []
    prev = -2
    for b in dots:
        if b > prev + 1:
            run_lengths.extend((int(b) + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def rle_encode(masks, fg_val=1):
    return ';'.join(json.dumps(_rle_one(m)) for m in masks)

In [ ]:
## Test prediction & submission 

if SUBMISSIONING:
    folder_path_test = Path("/kaggle/input/recodai-luc-scientific-image-forgery-detection/test_images")
    test_pred = {}
    
    for file_path in natsorted(folder_path_test.glob("*.png")):
        image_id = os.path.splitext(os.path.basename(file_path))[0]
        image = cv2.imread(str(file_path))
        orig_sizes = image[:,:,0:1]
        image = preprocess_images(image, False)
        image_tensor = tf.expand_dims(tf.convert_to_tensor(image), 0)
        test_probs = model.predict(image_tensor, verbose=0)[0]
        test_preds = (test_probs>0.5).astype(dtype=np.uint8)*255
        pred_np = postprocess_images(test_preds, orig_sizes)[:,:,0].numpy()
        pred_np = (pred_np>122).astype(dtype=np.uint8)
        if pred_np.sum() == 0:
            test_pred[image_id] = "authentic"
        else:
            rle = rle_encode([pred_np], fg_val=1)
            test_pred[image_id] = rle
    
    submission_df = pd.DataFrame([{"case_id": k, "annotation": v} for k, v in test_pred.items()])
    submission_df.to_csv("submission.csv", index=False)
    print("✅ submission.csv saved!")
    print(submission_df.head())   

# 8. Experimental code (e.g. for debugging)

In [ ]:
# ## Plot learning curves for definition of start leraning rate
# lrs = 1e-5 * (10 ** (np.arange(len(history.history["loss"])) / 10)) # Define the learning rate array
# plt.figure(figsize=(10, 6)) # Set the figure size
# plt.grid(True) # Set the grid
# plt.semilogx(lrs, history.history["loss"]) # Plot the loss in log scale
# plt.tick_params('both', length=10, width=1, which='both') # Increase the tickmarks size
# #plt.axis([1e-5, 1e-0, 0, 10]) # Set the plot boundaries

In [ ]:
# max_x = 0
# max_y = 0
# max_z = 0
# for image in labels_fo:
#     if image.shape[0] > max_x:
#         max_x = image.shape[0]
#     if image.shape[1] > max_y:
#         max_y = image.shape[1]  
#     if image.shape[2] > max_z:
#         max_z = image.shape[2]
# print(max_x)
# print(max_y)
# print(max_z)